<a href="https://colab.research.google.com/github/Jakelinecs/Tareas-Machine-Learning/blob/main/N34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
==================================================================================================
PRACTICO COMPLETO: TRADUCCIÓN AUTOMÁTICA Y SUBTITULADO DE IMÁGENES (KERAS/PYTORCH)
==================================================================================================

Este script contiene la implementación del modelo Seq2Seq (LSTM a nivel de carácter) para la
traducción automática (Punto 2) y la implementación de la Tarea Avanzada de Keras para el
Subtitulado de Imágenes (Punto 3), simulando la arquitectura del archivo model.py de PyTorch.

Nota: El código de traducción automática está completamente documentado línea por línea
para cumplir con el requisito de la tarea.
"""

import numpy as np
# Línea 39: Importa la librería NumPy, esencial para el manejo eficiente de arrays y tensores.
import keras
# Línea 40: Importa la librería Keras para construir y entrenar la red neuronal.
import os
# Línea 41: Importa el módulo 'os' para interactuar con el sistema operativo (comandos y rutas).
from pathlib import Path
# Línea 42: Importa la clase 'Path' de pathlib para manejar rutas de archivos.

# === LÍNEAS 5-34: INTRODUCCIÓN/SETUP (Texto original del script) ===
# (Estas líneas son comentarios informativos en el script original y se omiten aquí por ser prosa,
# pero en el archivo final deben estar presentes como el bloque de comentario introductorio original).
# Líneas 38-41: Bloque de importación.

# ------------------------------------------------------------------
# 2. TRADUCCIÓN AUTOMÁTICA (Implementación de lstm_seq2seq.py)
# ------------------------------------------------------------------

"""
## Descarga de datos
"""

fpath = keras.utils.get_file(origin="http://www.manythings.org/anki/fra-eng.zip")
# Línea 47: Descarga el archivo ZIP del corpus de traducción y almacena su ruta local en 'fpath'.
dirpath = Path(fpath).parent.absolute()
# Línea 48: Obtiene el directorio absoluto donde se descargó el archivo ZIP.
os.system(f"unzip -q {fpath} -d {dirpath}")
# Línea 49: Ejecuta un comando del sistema para descomprimir silenciosamente (-q) el archivo.

"""
## Configuración
"""

batch_size = 64  # Batch size for training.
# Línea 54: Define el tamaño del lote (batch size) para el entrenamiento.
epochs = 100  # Number of epochs to train for.
# Línea 55: Define el número de épocas para el entrenamiento.
latent_dim = 256  # Latent dimensionality of the encoding space.
# Línea 56: Define la dimensionalidad latente (unidades internas) de las capas LSTM.
num_samples = 10000  # Number of samples to train on.
# Línea 57: Define el número máximo de pares de frases a utilizar.
# Path to the data txt file on disk.
data_path = os.path.join(dirpath, "fra.txt")
# Línea 59: Construye la ruta completa al archivo de datos de texto sin comprimir.

"""
## Preparación de datos
"""

# Vectorize the data.
# Línea 64: Comentario indicando el inicio de la fase de vectorización.
input_texts = []
# Línea 65: Inicializa lista para frases de entrada (Inglés).
target_texts = []
# Línea 66: Inicializa lista para frases de destino (Francés).
input_characters = set()
# Línea 67: Inicializa conjunto para caracteres únicos de entrada.
target_characters = set()
# Línea 68: Inicializa conjunto para caracteres únicos de destino.
with open(data_path, "r", encoding="utf-8") as f:
# Línea 69: Abre el archivo de datos en modo lectura.
    lines = f.read().split("\n")
# Línea 70: Lee todo el archivo y lo divide en una lista de frases.
for line in lines[: min(num_samples, len(lines) - 1)]:
# Línea 71: Itera sobre las primeras 'num_samples' líneas.
    input_text, target_text, _ = line.split("\t")
# Línea 72: Divide la línea por el tabulador ('\t').
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
# Línea 76: Añade el token de INICIO DE SECUENCIA ('\t') y FIN DE SECUENCIA ('\n') a la frase de destino.
    input_texts.append(input_text)
# Línea 77: Agrega la frase de entrada a la lista.
    target_texts.append(target_text)
# Línea 78: Agrega la frase de destino (con tokens) a la lista.
    for char in input_text:
# Línea 79: Itera sobre los caracteres de la frase de entrada.
        if char not in input_characters:
# Línea 80: Verifica si el carácter es nuevo.
            input_characters.add(char)
# Línea 81: Agrega el carácter al conjunto de entrada.
    for char in target_text:
# Línea 82: Itera sobre los caracteres de la frase de destino.
        if char not in target_characters:
# Línea 83: Verifica si el carácter es nuevo.
            target_characters.add(char)
# Línea 84: Agrega el carácter al conjunto de destino.

input_characters = sorted(list(input_characters))
# Línea 86: Convierte y ordena el vocabulario de entrada.
target_characters = sorted(list(target_characters))
# Línea 87: Convierte y ordena el vocabulario de destino.
num_encoder_tokens = len(input_characters)
# Línea 88: Calcula el tamaño del vocabulario de entrada.
num_decoder_tokens = len(target_characters)
# Línea 89: Calcula el tamaño del vocabulario de destino.
max_encoder_seq_length = max([len(txt) for txt in input_texts])
# Línea 90: Calcula la longitud máxima de la secuencia de entrada.
max_decoder_seq_length = max([len(txt) for txt in target_texts])
# Línea 91: Calcula la longitud máxima de la secuencia de destino.

print("Number of samples:", len(input_texts))
# Línea 93: Imprime el número de muestras.
print("Number of unique input tokens:", num_encoder_tokens)
# Línea 94: Imprime el tamaño del vocabulario de entrada.
print("Number of unique output tokens:", num_decoder_tokens)
# Línea 95: Imprime el tamaño del vocabulario de destino.
print("Max sequence length for inputs:", max_encoder_seq_length)
# Línea 96: Imprime la longitud máxima de la secuencia de entrada.
print("Max sequence length for outputs:", max_decoder_seq_length)
# Línea 97: Imprime la longitud máxima de la secuencia de destino.

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# Línea 99: Crea un diccionario para mapear cada carácter de entrada a su índice numérico.
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
# Línea 100: Crea un diccionario para mapear cada carácter de destino a su índice numérico.

encoder_input_data = np.zeros(
# Línea 102: Inicializa el array NumPy para los datos de entrada del Codificador (one-hot).
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
# Línea 103: Forma: (num_muestras, longitud_máx_entrada, tamaño_vocabulario_entrada).
    dtype="float32",
# Línea 104: Tipo de datos.
)
decoder_input_data = np.zeros(
# Línea 106: Inicializa el array NumPy para los datos de entrada del Decodificador.
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
# Línea 107: Forma: (num_muestras, longitud_máx_destino, tamaño_vocabulario_destino).
    dtype="float32",
)
decoder_target_data = np.zeros(
# Línea 109: Inicializa el array NumPy para los datos de SALIDA/OBJETIVO del Decodificador.
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
# Línea 110: Forma: (num_muestras, longitud_máx_destino, tamaño_vocabulario_destino).
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
# Línea 114: Inicia el bucle de vectorización one-hot.
    for t, char in enumerate(input_text):
# Línea 115: Itera sobre cada carácter de la frase de entrada.
        encoder_input_data[i, t, input_token_index[char]] = 1.0
# Línea 116: Codifica en 1.0 la posición del carácter actual en la entrada del codificador.
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
# Línea 117: Rellena el resto de la secuencia de entrada con padding (espacio).
    for t, char in enumerate(target_text):
# Línea 118: Itera sobre cada carácter de la frase de destino.
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
# Línea 121: Codifica la entrada del decodificador (incluye '\t').
        if t > 0:
# Línea 122: Condicional para empezar a codificar el objetivo (salida) un paso después del '\t'.
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
# Línea 125: Codifica el OBJETIVO del decodificador, desplazado un paso atrás (t-1).
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
# Línea 126: Rellena el resto de la secuencia de entrada del decodificador con padding.
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0
# Línea 127: Rellena el resto de la secuencia objetivo del decodificador con padding.

"""
## Build the model
"""
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
# Línea 135: Define la capa de entrada del Codificador.
encoder = LSTM(latent_dim, return_state=True)
# Línea 136: Define la capa LSTM del Codificador, configurada para devolver solo los estados internos.
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Línea 137: Conecta la entrada a la LSTM y captura los estados h y c.

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Línea 140: Almacena los estados internos del Codificador (vector de contexto).

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
# Línea 143: Define la capa de entrada del Decodificador.

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# Línea 148: Define la capa LSTM del Decodificador, devuelve secuencias completas y estados.
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# Línea 149: Conecta la entrada y usa los estados del Codificador como estado inicial.
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
# Línea 150: Define la capa Densa de salida con activación softmax.
decoder_outputs = decoder_dense(decoder_outputs)
# Línea 151: Conecta la salida de la LSTM a la capa Densa.

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Línea 155: Crea el Modelo de Entrenamiento completo.

"""
## Train the model
"""

model.compile(
# Línea 160: Comienza la fase de compilación.
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
# Línea 161: Especifica el optimizador, la función de pérdida y la métrica.
)
# Nota: La ejecución de model.fit() se omite aquí para fines de compilación del archivo.
# model.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.2,
# )
# Save model
model.save("s2s_model.keras")
# Línea 171: Guarda el modelo entrenado en el disco.

"""
## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states
"""

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")
# Línea 184: Carga el modelo guardado.

encoder_inputs = model.input[0]  # input_1
# Línea 186: Define la entrada del Codificador de inferencia.
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
# Línea 187: Extrae los estados finales de la LSTM del Codificador.
encoder_states = [state_h_enc, state_c_enc]
# Línea 188: Almacena los estados.
encoder_model = Model(encoder_inputs, encoder_states)
# Línea 189: Define el MODELO CODIFICADOR DE INFERENCIA.

decoder_inputs = model.input[1]  # input_2
# Línea 191: Define la entrada del Decodificador de inferencia.
decoder_state_input_h = keras.Input(shape=(latent_dim,))
# Línea 192: Define el placeholder para el estado oculto (h) de entrada.
decoder_state_input_c = keras.Input(shape=(latent_dim,))
# Línea 193: Define el placeholder para el estado de celda (c) de entrada.
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# Línea 194: Agrupa los estados de entrada del decodificador.
decoder_lstm = model.layers[3]
# Línea 195: Referencia a la capa LSTM del Decodificador.
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
# Línea 196: Conecta la entrada de la secuencia de 1 paso a la LSTM.
    decoder_inputs, initial_state=decoder_states_inputs
# Línea 197: Usa los placeholders de estados como el estado inicial.
)
decoder_states = [state_h_dec, state_c_dec]
# Línea 199: Los estados de salida de la LSTM (los estados actualizados).
decoder_dense = model.layers[4]
# Línea 200: Referencia a la capa Densa de salida.
decoder_outputs = decoder_dense(decoder_outputs)
# Línea 201: Conecta la salida de la LSTM a la capa Densa.
decoder_model = Model(
# Línea 202: Define el MODELO DECODIFICADOR DE INFERENCIA.
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
# Línea 203: Mapea la entrada (1 carácter + estados previos) a (1 carácter predicho + nuevos estados).
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
# Línea 209: Crea el diccionario inverso para mapear índices a caracteres de entrada.
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
# Línea 210: Crea el diccionario inverso para mapear índices a caracteres de destino.


def decode_sequence(input_seq):
# Línea 213: Define la función que ejecuta la traducción completa.
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)
# Línea 215: Usa el Codificador para obtener el vector de contexto (estados iniciales).

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
# Línea 218: Inicializa la secuencia de entrada del decodificador (tamaño 1).
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0
# Línea 220: Inserta el token de inicio de secuencia ('\t').

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
# Línea 223: Bandera de parada.
    decoded_sentence = ""
# Línea 224: Inicializa la cadena de texto de la traducción.
    while not stop_condition:
# Línea 225: Bucle de generación (un paso por carácter).
        output_tokens, h, c = decoder_model.predict(
# Línea 226: Usa el Decodificador para predecir el siguiente carácter.
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
# Línea 231: Encuentra el índice del carácter con la probabilidad más alta (Greedy search).
        sampled_char = reverse_target_char_index[sampled_token_index]
# Línea 232: Convierte el índice en el carácter real.
        decoded_sentence += sampled_char
# Línea 233: Agrega el carácter a la frase traducida.

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
# Línea 238: Condición de parada.
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
# Línea 241: Reinicia la secuencia de entrada del decodificador.
        target_seq[0, 0, sampled_token_index] = 1.0
# Línea 242: El carácter predicho se convierte en la entrada para el siguiente paso.

        # Update states
        states_value = [h, c]
# Línea 245: Actualiza los estados para el siguiente paso.
    return decoded_sentence
# Línea 246: Devuelve la frase traducida.


"""
You can now generate decoded sentences as such:
"""

for seq_index in range(20):
# Línea 252: Itera sobre las primeras 20 muestras.
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
# Línea 256: Extrae una sola secuencia de entrada.
    decoded_sentence = decode_sequence(input_seq)
# Línea 257: Llama a la función de decodificación.
    print("-")
# Línea 258: Separador visual.
    print("Input sentence:", input_texts[seq_index])
# Línea 259: Imprime la frase original.
    print("Decoded sentence:", decoded_sentence)
# Línea 260: Imprime la traducción generada.

# ------------------------------------------------------------------
# 3. SUBTITULADO DE IMÁGENES (Tarea Avanzada: Reescritura del modelo PyTorch en Keras)
# ------------------------------------------------------------------

from tensorflow.keras.layers import Embedding, Dropout, Concatenate, Lambda
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import backend as K

# --- Hyperparámetros basados en el archivo PyTorch model.py ---
EMBED_SIZE_IC = 256    # embed_size
HIDDEN_SIZE_IC = 512   # hidden_size
VOCAB_SIZE_IC = VOCAB_SIZE # Usamos el mismo vocabulario, pero debe ser el de captions.
RESNET_FC_IN_FEATURES = 2048 # Salida de ResNet152 antes del mapeo.

# =================================================================
# CODIFICADOR CNN (Simula EncoderCNN)
# =================================================================

def build_keras_encoder_ic():
    """Toma la imagen y extrae el vector de características."""

    # Keras espera (Alto, Ancho, Canales) para la imagen
    image_input = Input(shape=(224, 224, 3), name='ic_image_input')

    # ResNet152 sin la capa densa superior, con Global Average Pooling
    resnet = ResNet152(weights='imagenet', include_top=False, pooling='avg')

    # Extraer características de la CNN (shape=(None, 2048))
    cnn_features = resnet(image_input)

    # Capa Lineal (simula self.linear): mapea 2048 a EMBED_SIZE_IC (256)
    encoder_output = Dense(EMBED_SIZE_IC, activation='relu', name='ic_encoder_linear')(cnn_features)

    # Se podría añadir una capa BatchNormalization aquí para simular self.bn

    encoder_model = Model(inputs=image_input, outputs=encoder_output, name='IC_Encoder_CNN')

    return encoder_model, image_input, encoder_output

# =================================================================
# DECODIFICADOR RNN (Simula DecoderRNN)
# =================================================================

def build_keras_decoder_ic(feature_input_tensor):
    """Genera la descripción a partir de las características y la secuencia."""

    # 1. Entrada de la secuencia de palabras (captions)
    caption_inputs = Input(shape=(None,), name='ic_caption_input')

    # 2. Capa de Embedding (simula self.embed)
    embedding_layer = Embedding(VOCAB_SIZE_IC, EMBED_SIZE_IC, name='ic_decoder_embedding')
    embeddings = embedding_layer(caption_inputs)

    # 3. Concatenación (Simula torch.cat((features.unsqueeze(1), embeddings), 1))
    # La característica de la imagen actúa como el primer token/paso de tiempo.

    # Agrega la dimensión de tiempo a la característica de la imagen (Ej. de (256,) a (1, 256))
    feature_step = Lambda(lambda x: K.expand_dims(x, axis=1), name='ic_feature_step')(feature_input_tensor)

    # Concatena: [Feature_Image (Paso 0)] + [Embedded_Caption (Pasos 1 a N)]
    combined_input = Concatenate(axis=1, name='ic_combined_input')([feature_step, embeddings])

    # 4. Capa LSTM (simula self.lstm)
    # batch_first=True es el default en Keras.
    lstm_layer = LSTM(HIDDEN_SIZE_IC, return_sequences=True, name='ic_decoder_lstm')
    hiddens = lstm_layer(combined_input)

    # 5. Capa Lineal (simula self.linear)
    linear_output = Dense(VOCAB_SIZE_IC, activation='softmax', name='ic_decoder_output')
    outputs = linear_output(hiddens)

    return caption_inputs, outputs

# =================================================================
# ENSAMBLAJE DEL MODELO COMPLETO DE SUBTITULADO
# =================================================================

def create_full_captioning_model():
    """Ensambla el Codificador y el Decodificador."""

    encoder_model, image_input_tensor, feature_output_tensor = build_keras_encoder_ic()
    caption_input_tensor, decoder_output_tensor = build_keras_decoder_ic(feature_output_tensor)

    full_model = Model(
        inputs=[image_input_tensor, caption_input_tensor],
        outputs=decoder_output_tensor,
        name='Image_Captioning_Seq2Seq'
    )

    # Compilación (requerida para el entrenamiento)
    full_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # return full_model # Retorna el modelo de entrenamiento
    print("\n--- TAREA AVANZADA: MODELO DE SUBTITULADO DE IMÁGENES EN KERAS ---")
    print("Modelo completo de subtitulado de imágenes (simulación PyTorch model.py) creado.")
    full_model.summary()

# Ejecución para mostrar la arquitectura:
if __name__ == '__main__':
    # Esto ejecutaría la traducción automática (si model.fit no estuviera comentado)
    # Y luego mostraría el resumen del modelo de subtitulado.
    create_full_captioning_model()

In [3]:
"""
================================================================================================
3.2 INVESTIGACIÓN: EJECUCIÓN DEL MODELO DE SUBTITULADO EN KERAS Y MIGRACIÓN DE PESOS PYTORCH
================================================================================================

https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/03-advanced/image_captioning

Este bloque describe los pasos conceptuales necesarios para migrar un modelo entrenado en
PyTorch (como el de subtitulado de imágenes) para ser ejecutado y utilizado en Keras/TensorFlow.

--- PASOS CLAVE PARA LA MIGRACIÓN ---

1. DEFINIR LA ARQUITECTURA EQUIVALENTE EN KERAS:
   Se debe replicar la estructura del modelo de PyTorch (EncoderCNN + DecoderRNN)
   en Keras utilizando el Modelo Funcional. Las capas LSTM, Dense y Embedding de Keras
   deben tener exactamente las mismas dimensiones y configuraciones que las capas de PyTorch.

2. CARGA DE PESOS DE PYTORCH:
   Se utiliza PyTorch (torch.load) para cargar el archivo de pesos (.pth o .pt).
   Esto resulta en un diccionario de estados (state_dict) que contiene los tensores
   de peso por nombre de capa (ej., 'lstm.weight_ih_l0').

3. MAPEADO Y TRANSPOSICIÓN DE TENSORES (Paso Crítico):
   Keras/TensorFlow y PyTorch tienen convenciones diferentes para almacenar los pesos.

   A. CAPAS LINEALES/DENSAS (nn.Linear vs. keras.layers.Dense):
      - PyTorch almacena los pesos en el formato: [output_dim, input_dim].
      - Keras/TensorFlow almacena los pesos en el formato: [input_dim, output_dim].
      - ACCIÓN REQUERIDA: Los tensores de peso de PyTorch deben ser **TRANSPUESTOS** (usando .T o np.transpose)
        antes de ser asignados a la capa Densa de Keras.

        # Ejemplo conceptual:
        # keras_weights = pytorch_weights.transpose()
        # keras_biases = pytorch_biases (Los sesgos generalmente no necesitan transposición)

   B. CAPAS LSTM/GRU (nn.LSTM vs. keras.layers.LSTM):
      - Los tensores de LSTM (pesos de entrada, pesos recurrentes, sesgos) están concatenados
        de forma diferente en cada framework.
      - ACCIÓN REQUERIDA: Los tensores de PyTorch deben ser **divididos, reordenados y luego transpuestos**
        para que coincidan con el orden esperado por Keras: [kernel, recurrent_kernel, bias].

4. ASIGNACIÓN DE PESOS EN KERAS:
   Una vez que los tensores han sido convertidos y reestructurados, se inyectan en las
   capas correspondientes de Keras usando el método:

   # Ejemplo:
   # keras_layer = model.get_layer('nombre_de_la_capa_keras')
   # keras_layer.set_weights([converted_weights, converted_biases])

Este proceso garantiza que el modelo Keras utilice la inteligencia aprendida por el modelo PyTorch.
"""


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Lambda
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import backend as K

# --- Hyperparámetros Basados en el modelo PyTorch ---
EMBED_SIZE = 256    # Dimensión del embedding
HIDDEN_SIZE = 512   # Unidades de la LSTM
VOCAB_SIZE = 10000  # Tamaño del vocabulario (ejemplo, ajustar al real)

# =================================================================
# 1. CODIFICADOR CNN (Simula EncoderCNN)
# =================================================================

def build_keras_encoder_ic():
    """Define el Codificador que extrae el vector de características de la imagen."""

    # Entrada de la imagen (Ej. 224x224x3)
    image_input = Input(shape=(224, 224, 3), name='ic_image_input')

    # ResNet152 preentrenada, quitando la capa densa superior, usando Global Average Pooling
    resnet = ResNet152(weights='imagenet', include_top=False, pooling='avg')

    # Características de la CNN (Salida de pooling: 2048)
    cnn_features = resnet(image_input)

    # Capa Lineal (simula self.linear): Mapea 2048 características a EMBED_SIZE (256)
    # Este vector se usará como el primer token para el Decodificador.
    encoder_output = Dense(EMBED_SIZE, activation='relu', name='ic_encoder_linear')(cnn_features)

    return image_input, encoder_output

# =================================================================
# 2. DECODIFICADOR RNN (Simula DecoderRNN)
# =================================================================

def build_keras_decoder_ic(feature_input_tensor):
    """Define el Decodificador que genera la descripción secuencialmente."""

    # 1. Entrada de la secuencia de palabras (captions)
    caption_inputs = Input(shape=(None,), name='ic_caption_input')

    # 2. Capa de Embedding (simula self.embed)
    embedding_layer = Embedding(VOCAB_SIZE, EMBED_SIZE, mask_zero=True, name='ic_decoder_embedding')
    embeddings = embedding_layer(caption_inputs)

    # 3. Concatenación de Feature de Imagen como el primer token
    # Agrega la dimensión de tiempo a la característica de la imagen (de (256,) a (1, 256))
    feature_step = Lambda(lambda x: K.expand_dims(x, axis=1), name='ic_feature_step')(feature_input_tensor)

    # Concatena: [Feature_Image (Paso 0)] + [Embedded_Caption (Pasos 1 a N)]
    combined_input = Concatenate(axis=1, name='ic_combined_input')([feature_step, embeddings])

    # 4. Capa LSTM (simula self.lstm)
    # return_sequences=True para predecir un token en cada paso.
    lstm_layer = LSTM(HIDDEN_SIZE, return_sequences=True, name='ic_decoder_lstm')
    hiddens = lstm_layer(combined_input)

    # 5. Capa Lineal (simula self.linear)
    decoder_outputs = Dense(VOCAB_SIZE, activation='softmax', name='ic_decoder_output')(hiddens)

    return caption_inputs, decoder_outputs

# =================================================================
# 3. MODELO DE SUBTITULADO COMPLETO (Entrenamiento)
# =================================================================

image_input_tensor, feature_output_tensor = build_keras_encoder_ic()
caption_input_tensor, decoder_output_tensor = build_keras_decoder_ic(feature_output_tensor)

# Modelo completo: [Imagen, Caption_Input] -> [Caption_Output]
captioning_model = Model(
    inputs=[image_input_tensor, caption_input_tensor],
    outputs=decoder_output_tensor,
    name='Image_Captioning_Keras_Simulation'
)

# Ejemplo de Compilación:
# captioning_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# captioning_model.summary()

In [3]:
"""
================================================================================================
4. TAREA AVANZADA: INVESTIGACIÓN AVANZADA
================================================================================================

Este bloque contiene las respuestas a las preguntas de investigación avanzada,
estructuradas como documentación dentro del archivo Python.
"""

# ------------------------------------------------------------------
# 4.1. Traducción entre japonés e inglés (Jp <-> En)
# ------------------------------------------------------------------

"""
PREGUNTA: ¿Qué pasos se tomarían para traducir entre japonés e inglés?

El japonés (SOV: Sujeto-Objeto-Verbo) y el inglés (SVO: Sujeto-Verbo-Objeto)
son tipológicamente muy diferentes, lo que requiere métodos avanzados de tokenización
y reordenamiento.
"""

def pasos_traduccion_jp_en():
    """Describe los pasos clave para la traducción Japonés <-> Inglés."""

    # 1. Tokenización (Paso Crítico)
    print("1. Tokenización por Sub-Palabra:")
    # El japonés no usa espacios, por lo que la tokenización debe ser morfológica.
    # Se usarían algoritmos como SentencePiece o WordPiece para segmentar el texto
    # japonés en unidades (sub-palabras) que manejen los diferentes alfabetos (Kanji, Hiragana, Katakana).

    # 2. Reordenamiento Sintáctico
    print("2. Reordenamiento Sintáctico (SOV <-> SVO):")
    # Se requiere un modelo avanzado (Transformer o NMT con Atención) capaz de aprender
    # las complejas reglas para reordenar la estructura de la frase (por ejemplo, mover
    # el verbo al final en japonés y al centro en inglés).

    # 3. Corpus
    print("3. Uso de Corpus Paralelo Extenso:")
    # El entrenamiento requiere un corpus paralelo de gran calidad y tamaño (ej. ASPEC)
    # para que el modelo aprenda las reglas de mapeo de alta complejidad.

# ------------------------------------------------------------------
# 4.2. Métodos avanzados de traducción automática
# ------------------------------------------------------------------

def metodos_avanzados_traduccion():
    """Explora los métodos más allá del Seq2Seq básico."""

    # A. Traducción Automática Neuronal basada en Atención (NMT con Attention)
    print("\nA. NMT basado en Atención:")
    # Mecanismo: El decodificador calcula un vector de atención, un promedio ponderado
    # de TODOS los estados ocultos del codificador.
    # Ventaja: Permite al modelo 'enfocarse' dinámicamente en las partes relevantes
    # de la frase de origen mientras traduce cada palabra de salida. Esto resuelve
    # el 'cuello de botella' del Seq2Seq básico.

    # B. Modelos Transformer (El Estado del Arte)
    print("B. Modelos Transformer:")
    # Arquitectura: Abandonan las RNNs (LSTM/GRU) por completo. Se basan únicamente
    # en el mecanismo de Auto-Atención (Self-Attention) y Atención Multi-Head.
    # Ventaja: Permite el procesamiento en paralelo de la secuencia completa,
    # acelerando dramáticamente el entrenamiento y capturando mejor las dependencias
    # a larga distancia.

# ------------------------------------------------------------------
# 4.3. Generación de imágenes a partir de texto
# ------------------------------------------------------------------

def generacion_imagenes_texto():
    """Investiga la tecnología inversa al subtitulado de imágenes."""

    # El campo es dominado por los Modelos de Difusión.

    # 1. Modelos de Difusión (Diffusion Models)
    print("\n1. Modelos de Difusión (Estado del Arte):")
    # Mecanismo: El modelo se entrena para revertir un proceso progresivo de
    # adición de ruido (denoising).
    # Condicionamiento: El prompt de texto se inyecta (codificado, ej., usando CLIP)
    # como una condición en cada paso del proceso de 'denoising', guiando la
    # reconstrucción de la imagen para que coincida con la descripción.
    # Ejemplos: Stable Diffusion, DALL-E 2.

    # 2. Redes Generativas Adversarias Condicionadas (GANs)
    print("2. GANs Condicionadas:")
    # Mecanismo: Dos redes compiten. La Generadora crea la imagen a partir del texto,
    # y la Discriminadora juzga si la imagen es realista y si coincide con la descripción textual.

# =================================================================
# EJECUCIÓN (Llamar a las funciones para ver la documentación)
# =================================================================

if __name__ == '__main__':
    print("--- 4.1. Traducción Japonés <-> Inglés ---")
    pasos_traduccion_jp_en()

    print("\n--- 4.2. Métodos Avanzados de Traducción Automática ---")
    metodos_avanzados_traduccion()

    print("\n--- 4.3. Generación de Imágenes a partir de Texto ---")
    generacion_imagenes_texto()